## Névterek és érvényességi kör (namespaces and variable scope)

In [ ]:
# Egy függvény belsejében definiált változók kifelé nem látszanak.

a = 0

def f(x):
    a = x + 1
    b = a + 1
    return b


print(a)
print(f(10))
print(a)

print(b)

In [ ]:
a = 0

def f(x):
    y = x + 1
    
    def g(z):
        return z + 100
    
    b = y + 1
    return g(b)


print(f(1))
print(a)

In [ ]:
a = 0

def f(a):
    a = a + 1
    
    def g(a):
        return a + 100
    
    a = a + 1
    return g(a)


print(f(1))
print(a)

In [ ]:
a = 0


def f(x):
    a += 1
    return x + 1

print(a)
print(f(0))
print(a)

In [ ]:
def outer(x):
    a = 1
    
    def inner(y):
        a = 2
        return y
    
    result = inner(x)
    return a

print(outer(0))

A fentiekben körbejártuk a névtér (namespace) és az érvényességi kör (scope) fogalmát. Aki kíváncsi a mélyebb részletekre, annak ajánlom a [RealPython](https://realpython.com/python-namespaces-scope/) megfelelő tutorialját.

Pythonban lehetséges a globális névtérben definiált változók értékének megváltoztatása a lokális névtérben. Ekkor nem a lokális névtérben keres az interpreter egy `a`-t, hanem a globálisban. Egy függvény belsejében definiált függvény is tudja módosítani a külső függvényben definiált lokális változót, ha nagyon muszáj. Erre szolgálnak a `global` és `nonlocal` kulcsszavak, azonban egyáltalán nem baj, ha ezeket a lehetőségeket **soha nem használjuk**.

Csak azért, mert egy nyelv megenged bizonyos lehetőségeket, az nem azt jelenti, hogy azokat mind-mind jó ötlet használni. :)

## Mátrixok szinguláris felbontása - képtömörítés 

### (Singular value decomposition - image compression)

In [ ]:
import matplotlib.image as im
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as lg

Legyen $A\in\mathbb{R}^{n\times m}$ egy tetszőleges mátrix. Ekkor léteznek olyan $U\in\mathbb{R}^{n\times n}$, $D\in\mathbb{R}^{n\times m}$ és $V\in\mathbb{R}^{m\times m}$ mátrixok, hogy

$$ 
A = UDV^*, 
$$ 

ahol $U$ és $V$ úgynevezett unitér mátrixok, azaz $U^*U = UU^* = I_n$ és $V^*V=VV^*=I_m$, valamint $D$ diagonális mátrix, azaz $d_{ij}=0$, ha $i\ne j$. 

$D$ főátlójában nemnegatív számok állnak, csökkenő sorrendben: $d_{ii} = \sigma_i$, $\sigma_1\geq\sigma_2\geq\ldots\geq\sigma_r > \sigma_{r+1} = \ldots = \sigma_{\min(n,m)} = 0$, ahol $r$ az $A$ mátrix rangja. Ezeket a $\sigma$ értékeket az $A$ mátrix szinguláris értékeinek nevezzük.

Legyen $k\in\mathbb{N}$ adott természetes szám, ahol $k\leq\text{rank}(A)\leq\min\{n, m\}$. Keressük azt az $A_k$ mátrixot, melynek rangja $k$ és $A_k$ a $k$-rangú mátrixok közül a legjobban közelíti $A$-t, azaz $A_k$ az alábbi minmalizációs probléma megoldása:

$$ 
\left|\left| A - B \right|\right|_F \to \min !\qquad \mbox{ ahol }\quad B\in\mathbb{R}^{n\times m}, \ \text{rank}(B) = k. 
$$ 

Itt $\left|\left| X \right|\right|_F^2$ az $X$ elemeinek négyzetösszegét jelöli.

Ennek a feladatnak a megoldása az $A$ mátrix SVD-felbontásából számolható ki. Ha $A = UDV^*$, akkor legyen $U_k$ az $U$ első $k$ oszlopa, legyen $V_k$ a $V$ első $k$ sora, $D_k$ pedig a $D$ első $k$ diagonálisbeli elemét tartalmazó diagonális mátrix; ekkor a fenti feladat megoldása az $A_k = U_kD_kV_k^*$ mátrix.

In [ ]:
image = im.imread("img/parrots.png")

In [ ]:
plt.figure(figsize=(16, 10))
plt.imshow(image)
plt.show()

In [ ]:
help(lg.svd)

In [ ]:
def compress(image, k):
    row, col, _ = image.shape
    
    U_r, d_r, V_r = np.linalg.svd(image[:, :, 0], full_matrices=True)
    U_g, d_g, V_g = np.linalg.svd(image[:, :, 1], full_matrices=True)
    U_b, d_b, V_b = np.linalg.svd(image[:, :, 2], full_matrices=True)

    image_red_approx = U_r[:, :k] @ np.diag(d_r[:k]) @ V_r[:k, :]
    image_green_approx = U_g[:, :k] @ np.diag(d_g[:k]) @ V_g[:k, :]
    image_blue_approx = U_b[:, :k] @ np.diag(d_b[:k]) @ V_b[:k, :]

    image_reconstructed = np.zeros((row, col, 3))
    image_reconstructed[:, :, 0] = image_red_approx
    image_reconstructed[:, :, 1] = image_green_approx
    image_reconstructed[:, :, 2] = image_blue_approx
    image_reconstructed[image_reconstructed < 0] = 0
    image_reconstructed[image_reconstructed > 1] = 1
    
    print(f"Compression rate: {3*(U_r[:, :k].nbytes + V_r[:k, :].nbytes + d_r[:k].nbytes) / image.nbytes}")
    return image_reconstructed

In [ ]:
def plot_image_pair(original, filtered):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 12))
    ax1.imshow(original, cmap=plt.get_cmap('gray'))
    ax2.imshow(filtered, cmap=plt.get_cmap('gray'))
    fig.tight_layout()
    plt.show()

In [ ]:
plot_image_pair(image, compress(image, k=1))

In [ ]:
plot_image_pair(image, compress(image, k=5))

In [ ]:
plot_image_pair(image, compress(image, 10))

In [ ]:
plot_image_pair(image, compress(image, 25))

In [ ]:
plot_image_pair(image, compress(image, 50))

In [ ]:
plot_image_pair(image, compress(image, 80))

In [ ]:
plot_image_pair(image, compress(image, 100))